In [5]:
from tensorflow.python.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [26]:
print("[INFO] loading model and label binarizer...")
model = load_model("model/activity.model")
lb = pickle.loads(open("model/lb.pickle", "rb").read())
 
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)

[INFO] loading model and label binarizer...


In [27]:
vs = cv2.VideoCapture("example_clips/lifting.mp4")
writer = None
(W, H) = (None, None)

In [28]:
while True:
    (grabbed, frame) = vs.read()
    
    if not grabbed:
        break

    if W is None or H is None:
        (H, W) = frame.shape[:2]

    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)

    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]

    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
        writer = cv2.VideoWriter("output/lifting_128frame.avi", fourcc, 30, (W, H), True)

    writer.write(output)
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break